In [15]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types 
from pyspark.sql import functions as F

In [16]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [17]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-12 18:15:04|  2020-01-12 18:19:52|                 N|         1|          41|          41|              1|         0.78|        5.5|  0.0|    0.

In [18]:
# Register the DataFrames as a SQL temporary view
df_green.registerTempTable('green')
df_yellow.registerTempTable('yellow')

/home/lulu/spark/spark-3.5.0-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [19]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM
    green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2 
""")

In [20]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7|            769.73|            45|
|2020-01-01 00:00:00|  17|195.02999999999997|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|129.95999999999998|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|175.67000000000002|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [21]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/02/13 15:39:49 WARN MemoryManager: Total allocation exceeds 95.

In [23]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM
    yellow
WHERE 
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2 
""")

In [24]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')

24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/02/13 15:40:01 WARN MemoryManager: Total allocation exceeds 95.

## NOW JOIN THE TWO TABLES ON HOUR AND ZONE 

In [32]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')
    
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [35]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on = ['hour', 'zone'], how='outer')

In [36]:
df_join.write.parquet('data/report/revenue/join', mode='overwrite')

24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/02/13 15:48:40 WARN MemoryManager: Total allocation exceeds 95.

In [37]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              NULL|                 NULL|
|2020-01-01 00:00:00|  60|            160.04|                   6|57.620000000000005|                    2|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|            199.49|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  71|              23.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  78|             34.46|                   2|              NULL|                 NULL|
|2020-01-01 00:00:00|  88|  

## Redo the Join based on rematerialized data from the original load 
rather than recreating it. 

In [38]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [39]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')
    
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [40]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on = ['hour', 'zone'], how='outer')

In [41]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              NULL|                 NULL|
|2020-01-01 00:00:00|  60|            160.04|                   6|57.620000000000005|                    2|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|            199.49|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  71|              23.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  78|             34.46|                   2|              NULL|                 NULL|
|2020-01-01 00:00:00|  88|  

## Case where one table is big and the other is small


In [43]:
df_join = spark.read.parquet('data/report/revenue/join')

In [48]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [46]:
df_zones = spark.read.parquet('zones/', header=True)

In [47]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [49]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [55]:
df_result = df_result.drop('LocationID','zone')
df_result.show() 

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|              NULL|                NULL|1214.8000000000002|                   56|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              NULL|                NULL|10773.360000000004|                  455|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|143.77999999999997|                   4|             35.51|                    2| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|            133.35|                   7|              NULL|                 NULL|   Queens|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL| 4011.449999999998|                  188|Manhattan| Yello

In [56]:
df_result.write.parquet('data/tmp/revenue-zones', mode='overwrite')

24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/02/13 16:45:19 WARN MemoryManager: Total allocation exceeds 95.